In [ ]:
from transformers import BartTokenizer
caption="a field of grass and sunflowers"
tokenizer=BartTokenizer.from_pretrained('facebook/bart-large-cnn')
inputs=tokenizer(caption, return_tensors="pt",max_length=255,padding="max_length")
inputs=inputs.to(device)

In [ ]:
output_indices=model.generate(**inputs, do_sample=True, num_beams=1)


In [ ]:
import matplotlib.pyplot as plt
z_q = model.quantize.embedding(output_indices).reshape(1, 16, 16, 256).permute(0,3,1,2)
u=model.decode(z_q).add(1).div(2).cpu().squeeze().permute(1, 2, 0)
plt.imshow(u)